In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.utils import save_image
from PIL import Image

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)

        # Calculate the input size for the first linear layer dynamically
        self.fc1_input_size = 32 * 8 * 8  # Adjust based on the actual output size of the preceding layer
        self.fc1 = nn.Linear(self.fc1_input_size, 64)
        self.fc2 = nn.Linear(64, 2)  # Output: 2 classes (cat and dog)


In [12]:
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [13]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Adjust the size as needed
    transforms.ToTensor(),
])

train_dataset = ImageFolder(root=r"C:\Users\Tejas Tyagi\Downloads\archive (2)\kagglecatsanddogs_3367a\PetImages", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

net = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [11]:
for epoch in range(5):  # Adjust the number of epochs
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')

torch.save(net.state_dict(), 'simple_cnn.pth')

NotImplementedError: Module [SimpleCNN] is missing the required "forward" function

In [ ]:
net = SimpleCNN()
net.load_state_dict(torch.load('simple_cnn.pth'))

net.eval()

test_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])
test_image = test_transform(
    Image.open(r"C:\Users\Tejas Tyagi\Downloads\archive (2)\kagglecatsanddogs_3367a\PetImages\Cat\9884.jpg")).unsqueeze(
    0)

with torch.no_grad():
    output = net(test_image)
    _, predicted_class = torch.max(output, 1)

class_names = train_dataset.classes
print(f'The model predicts the image as: {class_names[predicted_class.item()]}')